In [1]:
import numpy as np
import pandas as pd
from pyomo.environ import *

## Working and able to find a solution - simple model with 5 hour values!

In [46]:

model=ConcreteModel()
#i={'0','1','2','3','4'}
model.hours=Set(initialize=[0,1,2,3,4])
solar_production={0: 0, 1: 2, 2: 4, 3:8, 4: 5}
demand={0:10, 1:11, 2:12, 3:19, 4:21}
buy_price={0:1, 1:3, 2:5, 3:15, 4:20}
sell_price={0:1, 1:1, 2:2, 3:18, 4:15}
model.d=Param(model.hours, initialize=demand)
model.sp=Param(model.hours, initialize=solar_production)
model.buyp=Param(model.hours, initialize=buy_price)
model.sellp=Param(model.hours, initialize=sell_price)
model.soc_max=Param(model.hours, initialize=20)
model.soc_min=Param(model.hours, initialize=5)
model.u_cs=Var(model.hours, within=Binary)
model.u_ds=Var(model.hours, within=Binary)
model.battery_state=Var(model.hours, within=Binary)
#model.from_grid=Var(model.hours, within=PositiveReals)
#model.to_grid=Var(model.hours, within=PositiveReals)
cs=5
ds=7
lb={i:0 for i in model.hours}
ub={i:solar_production[i] for i in model.hours}
ub_grid={i:200 for i in model.hours}
def from_grid_limit(m,i):
    return(lb[i], ub_grid[i])
model.from_grid=Var(model.hours, bounds=from_grid_limit)
def var_lim(m,i):
    return (lb[i], ub[i])
model.solar_direct_use=Var(model.hours, domain=PositiveReals, bounds=var_lim)
def soc_lim(m,i):
    return(m.soc_min[i], m.soc_max[i])
model.soc=Var(model.hours, domain=PositiveReals, bounds=soc_lim)
def soc_cons(m,i):
    if i==0:
        return (m.soc[i] == m.u_cs[i]*cs-m.u_ds[i]*ds+10)
    else:
        return (m.soc[i] == m.u_cs[i]*cs-m.u_ds[i]*ds+m.soc[i-1])
model.soc_constraint=Constraint(model.hours, rule=soc_cons)
def solar_flow(m,i):
    return (m.sp[i]==m.u_cs[i]*cs+m.solar_direct_use[i])
model.solar_flow_cons=Constraint(model.hours, rule=solar_flow)
def energy_flow(m,i):
    return (m.d[i] == m.solar_direct_use[i] + ds*m.u_ds[i] + model.from_grid[i])
model.energy_cons=Constraint(model.hours, rule=energy_flow)
def battery_cons(m,i):
    return (m.u_cs[i]+m.u_ds[i]==m.battery_state[i])
model.battery_cons=Constraint(model.hours, rule=battery_cons)
def obj_fn(m):
    return sum((m.from_grid[i])*m.buyp[i] for i in m.hours)
model.of=Objective(rule=obj_fn, sense=minimize)
solvername='glpk'
solverpath_folder='C:\\Data\\w64' 
solverpath_exe='C:\\Data\\w64\\glpsol' 
solver=SolverFactory(solvername,executable=solverpath_exe)
result=solver.solve(model)
print(f"The objective function result is: {value(model.of)} €")

The objective function result is: 497.0 €


In [47]:
model.display()

Model unknown

  Variables:
    u_cs : Size=5, Index=hours
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
    u_ds : Size=5, Index=hours
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
    battery_state : Size=5, Index=hours
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :   

## Simple model for test and tries!! Start here if needed to change naything else!

In [37]:

model=ConcreteModel()
solar_production=2
demand=10
buy_price=50
sell_price=20
soc_p=10
cs=5
ds=5
model.bat_state=Var(within=Binary)
model.u_cs=Var(within=Binary)
model.u_ds=Var(within=Binary)
model.from_grid=Var(bounds=(0,200))
model.sol_direct=Var(bounds=(0,solar_production))
model.soc=Var(domain=IntegerSet)
model.soc_limitu=Constraint(expr=model.soc<=20)
model.soc_limitl=Constraint(expr=model.soc>=5)
model.weight=Constraint(expr=(cs*model.u_cs-ds*model.u_ds+soc_p)==model.soc)
model.csds=Constraint(expr=(model.u_cs+model.u_ds)==model.bat_state)
model.batteryflow=Constraint(expr=(solar_production==model.u_cs*cs+model.sol_direct))
model.energy_flow=Constraint(expr=(model.sol_direct+model.u_ds*ds+model.from_grid)==demand)
model.of=Objective(expr=(model.from_grid*buy_price), sense=minimize)
solvername='glpk'
solverpath_folder='C:\\Data\\w64' 
solverpath_exe='C:\\Data\\w64\\glpsol' 
solver=SolverFactory(solvername,executable=solverpath_exe)
result=solver.solve(model)
model.of.display()

of : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 150.0


In [38]:
model.display()

Model unknown

  Variables:
    bat_state : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    u_cs : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :     1 : False : False : Binary
    u_ds : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    from_grid : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   3.0 :   200 : False : False :  Reals
    sol_direct : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   2.0 :     2 : False : False :  Reals
    soc : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   5.0 :  None : False : False : Integers

  Objectives:
    of : Size=1,

## A bit more complex model with pandas dataframe. Not working!!

In [50]:
#A bit more complex model with pandas dataframe. Not working
def opt(dem1,solpro1,buy_price1,sell_price_1):
    model=ConcreteModel()
    #i={'0','1','2','3','4'}
    model.hours=Set(initialize=range(24))
    solar_production={i: solpro1[i] for i in range(24)}
    demand={i: dem1[i] for i in range(24)}
    buy_price={i: buy_price1[i] for i in range(24)}
    sell_price={i: sell_price_1[i] for i in range(24)}
    model.d=Param(model.hours, initialize=demand)
    model.sp=Param(model.hours, initialize=solar_production)
    model.buyp=Param(model.hours, initialize=buy_price)
    model.sellp=Param(model.hours, initialize=sell_price)
    model.soc_max=Param(model.hours, initialize=20)
    model.soc_min=Param(model.hours, initialize=5)
    model.u_cs=Var(model.hours, within=Binary)
    model.u_ds=Var(model.hours, within=Binary)
    model.battery_state=Var(model.hours, within=Binary)
    #model.from_grid=Var(model.hours, within=PositiveReals)
    #model.to_grid=Var(model.hours, within=PositiveReals)
    cs=5
    ds=7
    lb={i:0 for i in model.hours}
    ub={i:solar_production[i] for i in model.hours}
    ub_grid={i:200 for i in model.hours}
    def from_grid_limit(m,i):
        return(lb[i], ub_grid[i])
    model.from_grid=Var(model.hours, bounds=from_grid_limit)
    def var_lim(m,i):
        return (lb[i], ub[i])
    model.solar_direct_use=Var(model.hours, domain=PositiveReals, bounds=var_lim)
    def soc_lim(m,i):
        return(m.soc_min[i], m.soc_max[i])
    model.soc=Var(model.hours, domain=PositiveReals, bounds=soc_lim)
    def soc_cons(m,i):
        if i==0:
            return (m.soc[i] == m.u_cs[i]*cs-m.u_ds[i]*ds+10)
        else:
            return (m.soc[i] == m.u_cs[i]*cs-m.u_ds[i]*ds+m.soc[i-1])
    model.soc_constraint=Constraint(model.hours, rule=soc_cons)
    def solar_flow(m,i):
        return (m.sp[i]==m.u_cs[i]*cs+m.solar_direct_use[i])
    model.solar_flow_cons=Constraint(model.hours, rule=solar_flow)
    def energy_flow(m,i):
        return (m.d[i] == m.solar_direct_use[i] + ds*m.u_ds[i] + model.from_grid[i])
    model.energy_cons=Constraint(model.hours, rule=energy_flow)
    def battery_cons(m,i):
        return (m.u_cs[i]+m.u_ds[i]==m.battery_state[i])
    model.battery_cons=Constraint(model.hours, rule=battery_cons)
    def obj_fn(m):
        return sum((m.from_grid[i])*m.buyp[i] for i in m.hours)
    model.of=Objective(rule=obj_fn, sense=minimize)
    solvername='glpk'
    solverpath_folder='C:\\Data\\w64' 
    solverpath_exe='C:\\Data\\w64\\glpsol' 
    solver=SolverFactory(solvername,executable=solverpath_exe)
    result=solver.solve(model)
    print(f"The objective function result is: {value(model.of)} €")
    return model

In [51]:
input=pd.read_excel("input-1.xlsx")
for i in range(1):
    dem1=np.asarray(input.iloc[i*24:((i+1)*24),0])
    solpro1=np.asarray(input.iloc[i*24:((i+1)*24),1])
    buy_price1=np.asarray(input.iloc[i*24:((i+1)*24),2])
    sell_price1=np.asarray(input.iloc[i*24:((i+1)*24),3])
    model=opt(dem1,solpro1,buy_price1, sell_price1)
    

ERROR: evaluating object as numeric value: from_grid[0]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object from_grid[0]
ERROR: evaluating object as numeric value: of
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object from_grid[0]


ValueError: No value for uninitialized NumericValue object from_grid[0]

In [29]:
model.display()

Model unknown

  Variables:
    u_cs : Size=24, Index=hours
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
          9 :     0 :  None :     1 : False :  True : Binary
         10 :     0 :  None :     1 : False :  True : Binary
         11 :     0 :  None :     1 : False :  True : Binary
         12 :     0 :  None :     1 : False :  True : Binary
         13 :     0 :  None :     1 : False :  True : Binary
         14 :     0 :  No